In [46]:
import ccxt
import pprint
import pandas as pd
import numpy as np
import pandas_ta as ta
import yfinance
import vectorbt as vbt
from datetime import datetime
import backtrader as bt
import backtesting as bting

In [50]:
df = pd.DataFrame().ta.ticker("XRP-USD",period="1y",interval="1h")

In [53]:
CustomStrategy = ta.Strategy(
    name="WMA",
    ta=[
        {"kind": "rsi"},
        {"kind": "bbands", "length": 20, "col_names": ("BBL", "BBM", "BBU")},
    ]
)
df.ta.strategy(CustomStrategy)

Not enough col_names were specified : got 3, expected 5.


In [54]:
df

,Open,High,Low,Close,Volume,Dividends,Stock Splits,RSI_14,BBL_20_2.0,BBM_20_2.0,BBU_20_2.0,BBB_20_2.0,BBP_20_2.0
2020-12-13 13:00:00+00:00,0.518157,0.526960,0.518157,0.524461,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-13 14:00:00+00:00,0.524370,0.524370,0.518967,0.520697,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-13 15:00:00+00:00,0.520617,0.522990,0.513718,0.517003,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-13 16:00:00+00:00,0.517050,0.526049,0.517050,0.524062,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-13 17:00:00+00:00,0.524078,0.524436,0.519994,0.521815,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-13 10:00:00+00:00,0.828175,0.829913,0.822843,0.827761,0,0,0,46.855379,0.814334,0.834575,0.854816,4.850591,0.331677
2021-12-13 11:00:00+00:00,0.828025,0.831308,0.827039,0.829462,1273472,0,0,48.342608,0.814909,0.834805,0.854701,4.766605,0.365726
2021-12-13 12:00:00+00:00,0.829800,0.832042,0.823188,0.824264,33235584,0,0,44.265721,0.814534,0.834672,0.854810,4.825442,0.241573
2021-12-13 13:00:00+00:00,0.824068,0.826167,0.819183,0.825060,41430784,0,0,45.030572,0.813527,0.834020,0.854514,4.914392,0.281387


In [38]:
class BasicRsiStrategy(bting.Strategy):
    def init(self):
        self.rsi = self.I(ta.rsi, self.data.df.Close, length=14)

    def next(self):
        today = self.rsi[-1]
        yesterday = self.rsi[-2]

        # Crosses below 30 (oversold, time to buy)
        if yesterday > 30 and today < 30 and not self.position.is_long:
            self.buy()

        # Crosses above 70 (overbought, time to sell)
        elif yesterday < 70 and today > 70 and self.position.size > 0:
            self.position.close()

In [39]:
strategy = BasicRsiStrategy

data = df

bt = bting.Backtest(
    data=data,
    strategy=strategy,
    cash=1000,
    commission=.002,
    exclusive_orders=False,
    trade_on_close=False,
)

stats = bt.run()
print(stats)

# Creates a nice HTML file with graphs
bt.plot(
    smooth_equity=True,
    superimpose=True,
)

Start                     2020-12-13 00:00:00
End                       2021-12-13 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                   22.404372
Equity Final [$]                  2854.492892
Equity Peak [$]                   2920.663697
Return [%]                         185.449289
Buy & Hold Return [%]               61.942302
Return (Ann.) [%]                  184.632411
Volatility (Ann.) [%]              248.074681
Sharpe Ratio                         0.744261
Sortino Ratio                        7.478027
Calmar Ratio                          7.36746
Max. Drawdown [%]                  -25.060524
Avg. Drawdown [%]                   -9.451324
Max. Drawdown Duration      144 days 00:00:00
Avg. Drawdown Duration       28 days 00:00:00
# Trades                                    3
Win Rate [%]                        66.666667
Best Trade [%]                      92.606238
Worst Trade [%]                     -0.548706
Avg. Trade [%]                    

Row(id='2738', ...)